# MATH2504 Project 2 2022 Semester 2 Submission

[Assignment Instructions](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/project2.html)

Student names: Limao Chang, Tiarne Graves

In [1]:
include("simulation_script.jl")
include("test/scenarios.jl")

## Project Structure

In the root folder is `simulation_script.jl`, which imports and loads the `GeneralizedJacksonSim` module, as well as some other packages that are useful when working with `GeneralizedJacksonSim`.

The `src/` folder contains the source code for `GeneralizedJacksonSim`.

The `test/` folder contains tests which can be run in `test/runtests.jl`, as well as the four provided test scenarios, which can be found in `test/scenarios.jl`.

## Perspective Seminar

## Task 1

**Features of amusement park that are not captured by model**:
- Amusement park attractions usually serve multiple people simultaneously, whereas the model assumes jobs can only be serviced one at a time
- People may cut in line and skip queues, so in reality it is not really a queue in the first-in, first-out (FIFO) sense

**Model assumptions that are unrealistic for the application of a theme park**:
- Assuming that queues (and by extension the theme park) have infinite capacity
- Assuming that people will unconditionally stay in a queue until they get to the front of the line - they may leave because the queue is too long, they want to go to another attraction, etc.
- Assuming that the time taken to move between attractions or leave the theme park is instant, as travelling between attractions takes time
- Service duration is independent from all other services - if people are at the theme park together as a party (e.g. a family travelling around the theme park), their service durations will not be independent from each other
- Arrival of jobs to nodes is exponentially distributed - the arrival rate will likely be higher during the day (more traffic) than in the evening, for example.

**If you were actually using this model to assess congestion at the park, would it be useful or not?**

## Task 2

The total steady state mean queue lengths are plotted against $\rho$* below for the four scenarios.

In [ ]:
"""
    plot_total_ss_mean_queue_length(net::NetworkParameters)

Plots total steady state mean queue length as a function of ρ* for the given scenario.
"""
function plot_total_ss_mean_queue_length(net::NetworkParameters, scenario_number::Int64)
    ρ_star_values = 0.1:0.01:0.9
    total_ss_mean_queue_lengths = zeros(length(ρ_star_values))

    for (index, ρ_star) in enumerate(ρ_star_values)
        # adjust network parameters
        adjusted_scenario = set_scenario(net, ρ_star)
        ρ = compute_ρ(adjusted_scenario)

        total_ss_mean_queue_lengths[index] = sum(ρ ./ (1 .- ρ))
    end

    return plot(ρ_star_values,
                total_ss_mean_queue_lengths,
                xlabel="ρ",
                ylabel="Total Steady State Mean Queue Length",
                title="Scenario $scenario_number Theoretical")
end

p1 = plot_total_ss_mean_queue_length(scenario1, 1)
p2 = plot_total_ss_mean_queue_length(scenario2, 2)
p3 = plot_total_ss_mean_queue_length(scenario3, 3)
p4 = plot_total_ss_mean_queue_length(scenario4, 4)
        
plot(p1, p2, p3, p4, layout=(2, 2), legend=false, size=(800, 800))

## Task 3

**Simulation engine**: `src/GeneralizedJacksonSim.jl` contains the `GeneralizedJacksonSim` module, as well as the simulation engine used (`sim_net()` below).

```julia
# src/GeneralizedJacksonSim.jl
"""
A discrete event simulation engine for Open Generalized Jackson Networks.
"""
module GeneralizedJacksonSim

import Base: isless
using Accessors, DataStructures, Distributions, StatsBase, Parameters, LinearAlgebra,
    Random, Plots

include("network_parameters.jl")
include("state.jl")
include("event.jl")

export NetworkParameters, compute_ρ, maximal_alpha_scaling, set_scenario, sim_net

"""
Runs a discrete event simulation of an Open Generalized Jackson Network `net`.

The simulation runs from time `0` to `max_time`.

Statistics about the total mean queue lengths are recorded from `warm_up_time` onwards
and the estimated value is returned.

This simulation does NOT keep individual customers' state, it only keeps the state which is
the number of items in each of the nodes.
"""
function sim_net(net::NetworkParameters;
                 max_time=10^6, warm_up_time=10^4, seed::Int64=42)::Float64
    Random.seed!(seed)

    # create priority queue and add standard events
    priority_queue = BinaryMinHeap{TimedEvent}()
    for q in 1:net.L
        push!(priority_queue, TimedEvent(ExternalArrivalEvent(q), 0.0))
    end
    push!(priority_queue, TimedEvent(EndSimEvent(), max_time))

    # initialise state and time
    state = QueueNetworkState(zeros(Int64, net.L), net.L, net)
    time = 0.0

    # set up queues integral for computing total mean queue length
    queues_integral = zeros(net.L)
    last_time = 0.0

    """
    Records the queue integral of the given state at the given point in time.
    """
    function record_integral(time::Float64, state::State)
        (time >= warm_up_time) && (queues_integral += state.queues * (time - last_time))
        last_time = time
    end

    record_integral(time, state)

    # simulation loop
    while true
        # process the next upcoming event
        timed_event = pop!(priority_queue)
        time = timed_event.time
        new_timed_events = process_event(time, state, timed_event.event)

        isa(timed_event.event, EndSimEvent) && break

        # add new spawned events to queue
        for nte in new_timed_events
            push!(priority_queue, nte)
        end

        # record mean queue length
        record_integral(time, state)
    end

    return sum(queues_integral / max_time)
end

end  # end of module
```


The other functionalities of the simulation engine (`NetworkParameters`, `State`s, `Event`s, and the functions related to each of them) have been separated into their own files, to keep the code nice and structured. Below are some snippets from each file:

```julia
# src/network_parameters.jl
"""
    NetworkParameters

# Fields
- `L::Int`: the dimension of the network (number of nodes)
- `α_vector::Vector{Float64}`: the external arrival rates α_i >= 0
- `μ_vector::Vector{Float64}`: the service rates μ_i > 0
- `P::Matrix{Float64}`: the L×L routing matrix P
- `c_s::Float64`: squared coefficient of variation of the service processes, defaults to 1.0
"""
@with_kw struct NetworkParameters
    L::Int
    α_vector::Vector{Float64}
    μ_vector::Vector{Float64}
    P::Matrix{Float64}
    c_s::Float64 = 1.0
end

# src/state.jl
"""
    QueueNetworkState

# Fields
- `queues::Vector{Int}`: vector of number of customers in each queue
- `num_queues::Int`: number of queues, equivalent to `length(queues)`
- `net::NetworkParameters`: the parameters of the network
"""
mutable struct QueueNetworkState <: State
    queues::Vector{Int}
    num_queues::Int
    net::NetworkParameters
end

"""
    next_arrival_time(s::State, q::Int)

Generates the next arrival time for the `q`th server. The duration of time between
external arrivals is exponentially distributed with mean 1 / s.net.α_vector[q].
"""
next_arrival_time(s::State, q::Int) = rand(Exponential(1/s.net.α_vector[q]))

"""
    next_service_time(s::State, q::Int)

Generates the next service time for the `q`th server. The service duration is gamma
distributed.
"""
next_service_time(s::State, q::Int) = rand(rate_scv_gamma(s.net.μ_vector[q], s.net.c_s))

# src/event.jl
"""
    process_event(time::Float64, state::State, event::ExternalArrivalEvent)

Process an arrival event from outside the system, and spawns a list of events that occur as
a consequence of this arrival.

On arrival, if the server is free (no jobs in the buffer/queue), the job starts to receive
service. If the server is busy, the job queues for service and waits for its turn.

The time between external arrival events for a given server is exponentially distributed,
and the service duration is gamma distributed.
"""
function process_event(time::Float64, state::State, event::ExternalArrivalEvent)
    q = event.q
    state.queues[q] += 1  # add to queue
    new_timed_events = TimedEvent[]

    # prepare next external arrival for this particular server
    push!(new_timed_events,
        TimedEvent(ExternalArrivalEvent(q), time + next_arrival_time(state, q)))

    # start serving this job if it is the only one in the queue
    if state.queues[q] == 1
        push!(new_timed_events,
            TimedEvent(EndOfServiceEvent(q), time + next_service_time(state, q)))
    end
    return new_timed_events
end

"""
    process_event(time::Float64, state::State, event::EndOfServiceEvent)

Process an end-of-service event, and spawns a list of events that occur as a consequence of
this end of service.

When a job completes service at a buffer, it either leaves the system, or moves to another
buffer (both happen immediately). After completing service in server i, a job moves to
server j with probability P[i, j], where P is the routing matrix.
"""
function process_event(time::Float64, state::State, event::EndOfServiceEvent)
    q = event.q
    state.queues[q] -= 1  # remove from queue
    @assert state.queues[q] >= 0
    new_timed_events = TimedEvent[]

    # if there is another customer in the queue, start serving them
    if state.queues[q] > 0
        service_time = next_service_time(state, q)
        push!(new_timed_events, TimedEvent(EndOfServiceEvent(q), time + service_time))
    end

    # simulate the next location for this job; indices 1:L are the probabilities of moving
    # to another server in the system, and the last index is the probability of exiting
    # the system
    L = state.net.L
    next_loc_weights = state.net.P[q, :]
    push!(next_loc_weights, 1 - sum(next_loc_weights))
    @assert sum(next_loc_weights) == 1
    next_loc = sample(1:L+1, Weights(next_loc_weights))

    if next_loc <= L
        # job is staying in the system
        state.queues[next_loc] += 1

        # start serving job if it is the only one in the queue
        if state.queues[next_loc] == 1
            service_time = next_service_time(state, next_loc)
            push!(new_timed_events,
                TimedEvent(EndOfServiceEvent(next_loc), time + service_time))
        end
    end
    return new_timed_events
end
```

**Test 1**: the code for test 1 can be found in `test/task3_test1.jl`. We decided to advantage of multi-threading for this test, since each simulation can be run independently, and their results can be stored without affecting any other simulations (see the source code for more details). To make use of multi-threading, you have to run Julia with the `-t` or `--threads` option, and specify the number of threads you want to utilise, e.g. `julia -t 8` to run Julia with 8 threads. The test can then by executed using:
```julia
task3_test1(
    [scenario1, scenario2, scenario3, scenario4],  # vector of all scenarios to test
    verbose=false,  # whether to print out progress messages as the simulations are run
    multithreaded=true  # whether to utilise multi-threading (julia needs to be run with
                        # `julia -t n` where n is the number of threads)
)
```

An example output is given below, for `max_time` ranging from 10^3 to 10^5 (beyond 10^5 is unreasonable for scenario 4 - it took around half an hour to run scenario 4 with `max_time = 10^6`). In every scenario, the absolute relative errors seem to decrease as $\rho^*$ gets larger, and also as `max_time` gets larger.

<center>
    <img src="img/task3test1.png" width="640" />
</center>

**Test 2**: the code for test 2 can be found in `test/task3_test2.jl`. In order to record the number of arrivals $A_i(t)$, we added an `arrivals` field to the `QueueNetworkState` struct. However, we need to be able to retrieve this `state` from outside the `sim_net()` function to calculate the simulated mean number of arrivals (we could also just have `sim_net()` return the mean number of arrivals as well, but this breaks compatibility with previous tests). This was done by adding `state` as a keyword argument to `sim_net()`:

```julia
# src/GeneralizedJacksonSim.jl
function sim_net(net::NetworkParameters;
                 state::State=QueueNetworkState(net), max_time::Int64=10^6,
                 warm_up_time::Int64=10^4, seed::Int64=42)::Float64
    
# src/state.jl
mutable struct QueueNetworkState <: State
    queues::Vector{Int}
    arrivals::Vector{Int}
    num_queues::Int
    net::NetworkParameters

    # Inner constructor for a given scenario's parameters
    function QueueNetworkState(net::NetworkParameters)
        new(zeros(Int64, net.L), zeros(Int64, net.L), net.L, net)
    end
end
```

Now if we pass in a `state` to `sim_net()`, we now have a reference to the final state of the simulation when it ends:

```julia
# test/task3_test2.jl
function task3_test2(scenarios::Vector{NetworkParameters};
                     max_time::Int64=10^6, digits::Int64=3,
                     c_s_values::Vector{Float64}=[0.1, 0.5, 1.0, 2.0, 4.0])
    for (index, scenario) in enumerate(scenarios)
        println("Scenario $index:")
        for c_s in c_s_values
            println("   c_s = $c_s:")
            scenario = @set scenario.c_s = c_s
            # pass in a state to sim_net() so we have a reference to the state to calculate the
            # mean number of arrivals
            state = QueueNetworkState(scenario)
            _ = sim_net(scenario, state=state, max_time=max_time)

            # calculate mean number of arrivals
            mean_num_arrivals = state.arrivals ./ max_time
            theoretical_mean_num_arrivals = (I - scenario.P') \ scenario.α_vector
            sum_of_squares = sum((mean_num_arrivals - theoretical_mean_num_arrivals).^2)

            println("       Simulated average number of arrivals  : " *
                    "$(round.(mean_num_arrivals, digits=digits))")
            println("       Theoretical average number of arrivals: " *
                    "$(round.(theoretical_mean_num_arrivals, digits=digits))")
            println("       Sum of squared differences            : " *
                    "$(round.(sum_of_squares, digits=digits))")
        end
    end
end
```

Running the test for all 4 scenarios gives
```
Scenario 1:
   c_s = 0.1:
       Simulated average number of arrivals  : [0.498, 0.498, 0.498]
       Theoretical average number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences            : 0.0
   c_s = 0.5:
       Simulated average number of arrivals  : [0.497, 0.497, 0.497]
       Theoretical average number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences            : 0.0
   c_s = 1.0:
       Simulated average number of arrivals  : [0.497, 0.497, 0.497]
       Theoretical average number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences            : 0.0
   c_s = 2.0:
       Simulated average number of arrivals  : [0.497, 0.497, 0.497]
       Theoretical average number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences            : 0.0
   c_s = 4.0:
       Simulated average number of arrivals  : [0.5, 0.5, 0.5]
       Theoretical average number of arrivals: [0.5, 0.5, 0.5]
       Sum of squared differences            : 0.0
Scenario 2:
   c_s = 0.1:
       Simulated average number of arrivals  : [0.709, 0.709, 0.709]
       Theoretical average number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences            : 0.0
   c_s = 0.5:
       Simulated average number of arrivals  : [0.714, 0.714, 0.714]
       Theoretical average number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences            : 0.0
   c_s = 1.0:
       Simulated average number of arrivals  : [0.713, 0.713, 0.713]
       Theoretical average number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences            : 0.0
   c_s = 2.0:
       Simulated average number of arrivals  : [0.72, 0.72, 0.72]
       Theoretical average number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences            : 0.0
   c_s = 4.0:
       Simulated average number of arrivals  : [0.713, 0.713, 0.713]
       Theoretical average number of arrivals: [0.714, 0.714, 0.714]
       Sum of squared differences            : 0.0
Scenario 3:
   c_s = 0.1:
       Simulated average number of arrivals  : [3.694, 1.801, 2.44, 2.95, 3.368]
       Theoretical average number of arrivals: [5.0, 5.0, 5.0, 5.0, 5.0]
       Sum of squared differences            : 25.357
   c_s = 0.5:
       Simulated average number of arrivals  : [3.682, 1.796, 2.433, 2.951, 3.357]
       Theoretical average number of arrivals: [5.0, 5.0, 5.0, 5.0, 5.0]
       Sum of squared differences            : 25.491
   c_s = 1.0:
       Simulated average number of arrivals  : [3.685, 1.806, 2.448, 2.959, 3.36]
       Theoretical average number of arrivals: [5.0, 5.0, 5.0, 5.0, 5.0]
       Sum of squared differences            : 25.301
   c_s = 2.0:
       Simulated average number of arrivals  : [3.694, 1.804, 2.444, 2.959, 3.371]
       Theoretical average number of arrivals: [5.0, 5.0, 5.0, 5.0, 5.0]
       Sum of squared differences            : 25.272
   c_s = 4.0:
       Simulated average number of arrivals  : [3.692, 1.794, 2.436, 2.944, 3.361]
       Theoretical average number of arrivals: [5.0, 5.0, 5.0, 5.0, 5.0]
       Sum of squared differences            : 25.482
Scenario 4:
   c_s = 0.1:
       Simulated average number of arrivals  : [1.596, 1.601, 1.58, 1.588, 1.562, 1.575, 1.569, 1.634, 1.542, 1.628, 1.57, 1.558, 1.524, 1.497, 1.651, 1.633, 1.624, 1.494, 1.547, 1.527, 1.576, 1.51, 1.572, 1.508, 1.549, 1.575, 1.574, 1.546, 1.577, 1.53, 1.632, 1.607, 1.544, 1.546, 1.492, 1.583, 1.541, 1.523, 1.523, 1.538, 1.634, 1.597, 1.638, 1.527, 1.468, 1.544, 1.517, 1.531, 1.524, 1.569, 1.566, 1.509, 1.574, 1.604, 1.645, 1.532, 1.561, 1.596, 1.563, 1.567, 1.548, 1.477, 1.549, 1.607, 1.539, 1.642, 1.533, 1.522, 1.555, 1.586, 1.548, 1.542, 1.513, 1.646, 1.55, 1.537, 1.53, 1.494, 1.553, 1.57, 1.53, 1.565, 1.629, 1.547, 1.617, 1.627, 1.581, 1.55, 1.503, 1.592, 1.522, 1.585, 1.67, 1.548, 1.625, 1.58, 1.582, 1.584, 1.569, 1.516]
       Theoretical average number of arrivals: [2.307, 2.307, 2.284, 2.302, 2.284, 2.271, 2.288, 2.413, 2.205, 2.36, 2.272, 2.304, 2.197, 2.1, 2.47, 2.402, 2.385, 2.131, 2.226, 2.141, 2.319, 2.134, 2.327, 2.136, 2.255, 2.291, 2.328, 2.244, 2.359, 2.227, 2.398, 2.348, 2.234, 2.19, 2.143, 2.321, 2.222, 2.169, 2.219, 2.226, 2.409, 2.328, 2.401, 2.191, 2.043, 2.269, 2.154, 2.194, 2.168, 2.226, 2.253, 2.136, 2.238, 2.343, 2.421, 2.201, 2.246, 2.346, 2.245, 2.233, 2.269, 2.077, 2.234, 2.355, 2.187, 2.431, 2.198, 2.15, 2.253, 2.295, 2.272, 2.196, 2.137, 2.495, 2.24, 2.203, 2.173, 2.144, 2.223, 2.301, 2.222, 2.294, 2.425, 2.228, 2.378, 2.389, 2.261, 2.23, 2.149, 2.326, 2.193, 2.333, 2.45, 2.228, 2.426, 2.319, 2.316, 2.346, 2.251, 2.152]
       Sum of squared differences            : 49.506
   c_s = 0.5:
       Simulated average number of arrivals  : [1.59, 1.609, 1.568, 1.589, 1.564, 1.574, 1.579, 1.635, 1.54, 1.631, 1.561, 1.564, 1.523, 1.495, 1.645, 1.627, 1.615, 1.498, 1.554, 1.52, 1.589, 1.512, 1.572, 1.509, 1.547, 1.573, 1.575, 1.542, 1.58, 1.534, 1.629, 1.608, 1.542, 1.544, 1.494, 1.575, 1.538, 1.516, 1.527, 1.545, 1.634, 1.59, 1.646, 1.531, 1.466, 1.545, 1.508, 1.539, 1.524, 1.563, 1.573, 1.504, 1.565, 1.598, 1.646, 1.529, 1.562, 1.598, 1.573, 1.571, 1.549, 1.476, 1.552, 1.602, 1.524, 1.647, 1.532, 1.52, 1.548, 1.587, 1.555, 1.538, 1.51, 1.648, 1.546, 1.537, 1.53, 1.493, 1.555, 1.577, 1.536, 1.564, 1.628, 1.557, 1.609, 1.632, 1.576, 1.549, 1.501, 1.593, 1.517, 1.606, 1.671, 1.55, 1.627, 1.583, 1.58, 1.582, 1.562, 1.509]
       Theoretical average number of arrivals: [2.307, 2.307, 2.284, 2.302, 2.284, 2.271, 2.288, 2.413, 2.205, 2.36, 2.272, 2.304, 2.197, 2.1, 2.47, 2.402, 2.385, 2.131, 2.226, 2.141, 2.319, 2.134, 2.327, 2.136, 2.255, 2.291, 2.328, 2.244, 2.359, 2.227, 2.398, 2.348, 2.234, 2.19, 2.143, 2.321, 2.222, 2.169, 2.219, 2.226, 2.409, 2.328, 2.401, 2.191, 2.043, 2.269, 2.154, 2.194, 2.168, 2.226, 2.253, 2.136, 2.238, 2.343, 2.421, 2.201, 2.246, 2.346, 2.245, 2.233, 2.269, 2.077, 2.234, 2.355, 2.187, 2.431, 2.198, 2.15, 2.253, 2.295, 2.272, 2.196, 2.137, 2.495, 2.24, 2.203, 2.173, 2.144, 2.223, 2.301, 2.222, 2.294, 2.425, 2.228, 2.378, 2.389, 2.261, 2.23, 2.149, 2.326, 2.193, 2.333, 2.45, 2.228, 2.426, 2.319, 2.316, 2.346, 2.251, 2.152]
       Sum of squared differences            : 49.528
   c_s = 1.0:
       Simulated average number of arrivals  : [1.592, 1.602, 1.571, 1.591, 1.567, 1.578, 1.571, 1.635, 1.536, 1.625, 1.557, 1.567, 1.524, 1.498, 1.643, 1.627, 1.616, 1.488, 1.556, 1.521, 1.58, 1.512, 1.571, 1.504, 1.549, 1.572, 1.579, 1.553, 1.584, 1.532, 1.637, 1.613, 1.548, 1.547, 1.496, 1.573, 1.539, 1.522, 1.521, 1.544, 1.636, 1.593, 1.642, 1.541, 1.471, 1.549, 1.513, 1.538, 1.534, 1.564, 1.577, 1.512, 1.561, 1.593, 1.639, 1.527, 1.568, 1.6, 1.57, 1.567, 1.548, 1.475, 1.547, 1.602, 1.531, 1.646, 1.537, 1.513, 1.553, 1.586, 1.561, 1.536, 1.516, 1.645, 1.547, 1.537, 1.523, 1.49, 1.56, 1.567, 1.532, 1.564, 1.633, 1.567, 1.615, 1.624, 1.584, 1.552, 1.504, 1.593, 1.524, 1.589, 1.674, 1.553, 1.612, 1.579, 1.583, 1.582, 1.564, 1.518]
       Theoretical average number of arrivals: [2.307, 2.307, 2.284, 2.302, 2.284, 2.271, 2.288, 2.413, 2.205, 2.36, 2.272, 2.304, 2.197, 2.1, 2.47, 2.402, 2.385, 2.131, 2.226, 2.141, 2.319, 2.134, 2.327, 2.136, 2.255, 2.291, 2.328, 2.244, 2.359, 2.227, 2.398, 2.348, 2.234, 2.19, 2.143, 2.321, 2.222, 2.169, 2.219, 2.226, 2.409, 2.328, 2.401, 2.191, 2.043, 2.269, 2.154, 2.194, 2.168, 2.226, 2.253, 2.136, 2.238, 2.343, 2.421, 2.201, 2.246, 2.346, 2.245, 2.233, 2.269, 2.077, 2.234, 2.355, 2.187, 2.431, 2.198, 2.15, 2.253, 2.295, 2.272, 2.196, 2.137, 2.495, 2.24, 2.203, 2.173, 2.144, 2.223, 2.301, 2.222, 2.294, 2.425, 2.228, 2.378, 2.389, 2.261, 2.23, 2.149, 2.326, 2.193, 2.333, 2.45, 2.228, 2.426, 2.319, 2.316, 2.346, 2.251, 2.152]
       Sum of squared differences            : 49.476
   c_s = 2.0:
       Simulated average number of arrivals  : [1.592, 1.604, 1.571, 1.588, 1.566, 1.57, 1.578, 1.631, 1.54, 1.633, 1.569, 1.567, 1.526, 1.496, 1.646, 1.628, 1.619, 1.502, 1.555, 1.523, 1.576, 1.514, 1.568, 1.508, 1.55, 1.57, 1.577, 1.547, 1.578, 1.536, 1.636, 1.617, 1.547, 1.538, 1.492, 1.569, 1.532, 1.516, 1.525, 1.541, 1.641, 1.599, 1.655, 1.532, 1.472, 1.55, 1.519, 1.541, 1.522, 1.57, 1.579, 1.507, 1.551, 1.597, 1.64, 1.531, 1.563, 1.597, 1.553, 1.573, 1.558, 1.476, 1.535, 1.602, 1.533, 1.639, 1.536, 1.518, 1.557, 1.583, 1.552, 1.545, 1.515, 1.654, 1.556, 1.538, 1.529, 1.508, 1.553, 1.577, 1.533, 1.565, 1.634, 1.556, 1.609, 1.633, 1.582, 1.555, 1.501, 1.595, 1.523, 1.59, 1.664, 1.55, 1.615, 1.586, 1.582, 1.591, 1.568, 1.508]
       Theoretical average number of arrivals: [2.307, 2.307, 2.284, 2.302, 2.284, 2.271, 2.288, 2.413, 2.205, 2.36, 2.272, 2.304, 2.197, 2.1, 2.47, 2.402, 2.385, 2.131, 2.226, 2.141, 2.319, 2.134, 2.327, 2.136, 2.255, 2.291, 2.328, 2.244, 2.359, 2.227, 2.398, 2.348, 2.234, 2.19, 2.143, 2.321, 2.222, 2.169, 2.219, 2.226, 2.409, 2.328, 2.401, 2.191, 2.043, 2.269, 2.154, 2.194, 2.168, 2.226, 2.253, 2.136, 2.238, 2.343, 2.421, 2.201, 2.246, 2.346, 2.245, 2.233, 2.269, 2.077, 2.234, 2.355, 2.187, 2.431, 2.198, 2.15, 2.253, 2.295, 2.272, 2.196, 2.137, 2.495, 2.24, 2.203, 2.173, 2.144, 2.223, 2.301, 2.222, 2.294, 2.425, 2.228, 2.378, 2.389, 2.261, 2.23, 2.149, 2.326, 2.193, 2.333, 2.45, 2.228, 2.426, 2.319, 2.316, 2.346, 2.251, 2.152]
       Sum of squared differences            : 49.419
   c_s = 4.0:
       Simulated average number of arrivals  : [1.588, 1.61, 1.574, 1.585, 1.573, 1.572, 1.573, 1.629, 1.534, 1.622, 1.566, 1.562, 1.522, 1.497, 1.648, 1.622, 1.615, 1.493, 1.546, 1.52, 1.582, 1.516, 1.579, 1.503, 1.55, 1.565, 1.577, 1.545, 1.577, 1.536, 1.634, 1.618, 1.546, 1.55, 1.491, 1.577, 1.536, 1.514, 1.521, 1.537, 1.635, 1.595, 1.648, 1.531, 1.469, 1.545, 1.51, 1.533, 1.531, 1.568, 1.578, 1.509, 1.571, 1.591, 1.65, 1.529, 1.564, 1.595, 1.568, 1.568, 1.544, 1.479, 1.547, 1.6, 1.527, 1.644, 1.531, 1.512, 1.549, 1.581, 1.549, 1.546, 1.522, 1.644, 1.55, 1.542, 1.528, 1.496, 1.565, 1.579, 1.53, 1.566, 1.635, 1.549, 1.617, 1.642, 1.576, 1.551, 1.497, 1.591, 1.53, 1.584, 1.671, 1.552, 1.622, 1.577, 1.585, 1.588, 1.564, 1.51]
       Theoretical average number of arrivals: [2.307, 2.307, 2.284, 2.302, 2.284, 2.271, 2.288, 2.413, 2.205, 2.36, 2.272, 2.304, 2.197, 2.1, 2.47, 2.402, 2.385, 2.131, 2.226, 2.141, 2.319, 2.134, 2.327, 2.136, 2.255, 2.291, 2.328, 2.244, 2.359, 2.227, 2.398, 2.348, 2.234, 2.19, 2.143, 2.321, 2.222, 2.169, 2.219, 2.226, 2.409, 2.328, 2.401, 2.191, 2.043, 2.269, 2.154, 2.194, 2.168, 2.226, 2.253, 2.136, 2.238, 2.343, 2.421, 2.201, 2.246, 2.346, 2.245, 2.233, 2.269, 2.077, 2.234, 2.355, 2.187, 2.431, 2.198, 2.15, 2.253, 2.295, 2.272, 2.196, 2.137, 2.495, 2.24, 2.203, 2.173, 2.144, 2.223, 2.301, 2.222, 2.294, 2.425, 2.228, 2.378, 2.389, 2.261, 2.23, 2.149, 2.326, 2.193, 2.333, 2.45, 2.228, 2.426, 2.319, 2.316, 2.346, 2.251, 2.152]
       Sum of squared differences            : 49.517

```

## Task 4

For task 4, a similar code to task 2 was implemented. The full file for task for can be located in task4.jl. The code below generates the y coordinated for each service time at each p_star value. The function takes a specific network (i.e. scenario 1-4, so varing data can be generated). The array plotting_data stores 5 vectors, one for each service time.

In [ ]:
function plot_mean_queue_length_service_times(net::NetworkParameters, scenario_number::Int64; max_time=10^3, warm_up_time=10)
    ρ_star_values = 0.01:0.01:0.9
    service_time_values = [0.1,0.5,1.0,2.0,4.0]
    plotting_data = Vector{Vector{Float64}}()
#forming data for each service time
    for (j,service_time) in enumerate(service_time_values)
        simulated_total_mean_queue_lengths = zeros(length(ρ_star_values))
        #forming data for each p*
        for (index, ρ_star) in enumerate(ρ_star_values)
            # adjust network parameters
            adjusted_net = set_scenario(net, ρ_star, service_time)
            simulated = sim_net(adjusted_net, max_time=max_time, warm_up_time=warm_up_time)
            # get total steady state mean queue length
            simulated_total_mean_queue_lengths[index] = simulated
        end

        push!(plotting_data, simulated_total_mean_queue_lengths)
    end
    return plotting_data
end

To plot the required graphs the above function is called in the below function plotting_service_times, which allows the 5 varying service times for each scenario to be plotted on one figure, by extracting data from the plotting_data array.

In [ ]:
function plotting_service_times(net::NetworkParameters, scenario_number::Int64)
    ρ_star_values = 0.01:0.01:0.9
    data = plot_mean_queue_length_service_times(net, scenario_number,max_time=10^3, warm_up_time=10)
    return plot(ρ_star_values,[data[1],data[2],data[3],data[4],data[5]], xlabel="ρ",
        ylabel="Total Mean Queue Length",
        title="Scenario $scenario_number Simulation with varying service times",
        labels=["0.1" "0.5" "1.0" "2.0" "4.0"])
end

This results in the below graphs, it can be seen that higher service times e.g. 4.0 result in a higher line compared to lower service times e.g. 0.1

<center>
    <img src="img/task4_service_times.png" width="640" />
</center>

Confidence bounds for these curves were also generated using the code below, where 100 different points were generated for each p* by using a different seed for each generation as can be seen in the for loops below.

In [ ]:
function confidence_bounds(net::NetworkParameters, scenario_number::Int64, service_time::Float64,max_time=10^3, warm_up_time=10)
    ρ_star_values = 0.01:0.01:0.9
    data = []
    mean_data=[]
    top_percentile = []
    bottom_percentile = []
        for (index, ρ_star) in enumerate(ρ_star_values)
            adjusted_net = set_scenario(net, ρ_star, service_time)
            for seed in 1:100
                simulated = sim_net(adjusted_net, max_time=max_time, warm_up_time=warm_up_time,seed = seed)
                push!(data,simulated)
            end
        push!(mean_data, mean(data))
        push!(top_percentile, percentile(data,95))
        push!(bottom_percentile, percentile(data,5))
        end
return plot(ρ_star_values,mean_data; ribbon = (bottom_percentile,top_percentile), xlabel="ρ",
    ylabel="Total Mean Queue Length",
    title="Scenario $scenario_number Simulation with $service_time service time, confidence bound",
    labels=["mean" "95th percentile" "5th percentile"])
end

This function could then be called for each scenario and service time using the code below: 

In [ ]:
p5 = confidence_bounds(scenario1, 1, 0.1)
p6 = confidence_bounds(scenario1, 1, 0.5)
p7 = confidence_bounds(scenario1, 1, 1.0)
p8 = confidence_bounds(scenario1, 1, 2.0)
p9 = confidence_bounds(scenario1, 1, 4.0)

plot(p5,p6,p7,p8,p9,layout=(5, 1), legend=true, size=(1100, 1100))

The resulting confidence bounds for each scenario were as below:

<center>
    <img src="img/scenario1_confbounds.png" width="640" />
</center>

<center>
    <img src="img/scenario2_confbounds.png" width="640" />
</center>

<center>
    <img src="img/scenario3_confbound.png" width="640" />
</center>




## Task 5